# Keras generic implementation of tweetshashtag2vec

In [1]:
#import tensorflow backend
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import keras
from keras.layers import *
from keras.models import Sequential, Model
from keras.optimizers import *
from keras.preprocessing import sequence
print(keras.__version__)
print(keras.backend.backend())
import numpy as np

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17913481044391591244
]
2.1.2
tensorflow


Using TensorFlow backend.


In [2]:
#load data
with open("temp/neg_sample_data.json", "r", encoding="utf-8") as f:
    md_nfreq_sample = json.load(f)
    
with open("temp/tweets4classification.json", "r", encoding="utf-8") as f:
    md_freq_sample = json.load(f)

In [3]:
md_nfreq_sample['data'][0]

{'hashtag_label': [],
 'hashtags': ['cervicalhealthawarenessmonth', 'obamacar'],
 'id': '418769437261455360',
 'orignal_hashtags': ['#cervicalhealthawarenessmonth', '#obamacare'],
 'raw': 'rt @ppsne: jan. is #cervicalhealthawarenessmonth. thanks to #obamacare, pap tests + the hpv vaccine are covered without a copay. http://t.c…',
 'text': 'rt : jan. is cervicalhealthawarenessmonth. thanks to obamacare, pap tests + the hpv vaccine are covered without a copay.',
 'words': ['rt',
  ':',
  'jan.',
  'is',
  'cervicalhealthawarenessmonth',
  '.',
  'thanks',
  'to',
  'obamacare',
  ',',
  'pap',
  'tests',
  '+',
  'the',
  'hpv',
  'vaccine',
  'are',
  'covered',
  'without',
  'a',
  'copay',
  '.']}

In [4]:
md_freq_sample['data'][0]

{'hashtag_label': [1, 5],
 'hashtags': ['hpv', 'vaccin'],
 'id': '418263863772327936',
 'orignal_hashtags': ['#hpv', '#vaccine'],
 'raw': 'rt @cdcstd: #hpv vax coverage could be 93% if doctors gave hpv #vaccine each time a preteen/teen got any other vaccine&gt; http://t.co/xxryga5…',
 'text': 'rt : hpv vax coverage could be 93% if doctors gave hpv vaccine each time a preteen / teen got any other vaccine>',
 'words': ['rt',
  ':',
  'hpv',
  'vax',
  'coverage',
  'could',
  'be',
  '93',
  '%',
  'if',
  'doctors',
  'gave',
  'hpv',
  'vaccine',
  'each',
  'time',
  'a',
  'preteen',
  '/',
  'teen',
  'got',
  'any',
  'other',
  'vaccine',
  '>']}

# gensim implementation of tweetshashtag2vec using doc2vec api

In [6]:
import gensim

In [7]:
#data prepare
train_raw_data = md_freq_sample['data'] +  md_nfreq_sample['data']
print(train_raw_data[0]['words'])
print(len(train_raw_data))
train_raw_data[0]

['rt', ':', 'hpv', 'vax', 'coverage', 'could', 'be', '93', '%', 'if', 'doctors', 'gave', 'hpv', 'vaccine', 'each', 'time', 'a', 'preteen', '/', 'teen', 'got', 'any', 'other', 'vaccine', '>']
115165


{'hashtag_label': [1, 5],
 'hashtags': ['hpv', 'vaccin'],
 'id': '418263863772327936',
 'orignal_hashtags': ['#hpv', '#vaccine'],
 'raw': 'rt @cdcstd: #hpv vax coverage could be 93% if doctors gave hpv #vaccine each time a preteen/teen got any other vaccine&gt; http://t.co/xxryga5…',
 'text': 'rt : hpv vax coverage could be 93% if doctors gave hpv vaccine each time a preteen / teen got any other vaccine>',
 'words': ['rt',
  ':',
  'hpv',
  'vax',
  'coverage',
  'could',
  'be',
  '93',
  '%',
  'if',
  'doctors',
  'gave',
  'hpv',
  'vaccine',
  'each',
  'time',
  'a',
  'preteen',
  '/',
  'teen',
  'got',
  'any',
  'other',
  'vaccine',
  '>']}

In [8]:
from gensim.models.doc2vec import TaggedDocument
docs = [TaggedDocument(each['words'], each['orignal_hashtags']) for each in train_raw_data]
words_count = sum(len(each[0]) for each in docs)
tags_count = sum(len(each[1]) for each in docs)
print(words_count)
print(tags_count)
docs[0]

1960039
239984


TaggedDocument(words=['rt', ':', 'hpv', 'vax', 'coverage', 'could', 'be', '93', '%', 'if', 'doctors', 'gave', 'hpv', 'vaccine', 'each', 'time', 'a', 'preteen', '/', 'teen', 'got', 'any', 'other', 'vaccine', '>'], tags=['#hpv', '#vaccine'])

In [11]:
%%time 
#training model manully tune alpha
import multiprocessing
model = gensim.models.doc2vec.Doc2Vec(dm=0, dbow_words=1, size=100, alpha=0.025, min_alpha=0.025, min_count=1, workers=multiprocessing.cpu_count())
model.build_vocab(docs)

for epoch in range(100):
    print ('Now training epoch %s'%epoch)
    model.train(docs, total_examples = words_count, epochs = model.iter)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

Now training epoch 0
Now training epoch 1
Now training epoch 2
Now training epoch 3
Now training epoch 4
Now training epoch 5
Now training epoch 6
Now training epoch 7
Now training epoch 8
Now training epoch 9
Now training epoch 10
Now training epoch 11
Now training epoch 12
Now training epoch 13
Now training epoch 14
Now training epoch 15
Now training epoch 16
Now training epoch 17
Now training epoch 18
Now training epoch 19
Now training epoch 20
Now training epoch 21
Now training epoch 22
Now training epoch 23
Now training epoch 24
Now training epoch 25
Now training epoch 26
Now training epoch 27
Now training epoch 28
Now training epoch 29
Now training epoch 30
Now training epoch 31
Now training epoch 32
Now training epoch 33
Now training epoch 34
Now training epoch 35
Now training epoch 36
Now training epoch 37
Now training epoch 38
Now training epoch 39
Now training epoch 40
Now training epoch 41
Now training epoch 42
Now training epoch 43
Now training epoch 44
Now training epoch 4

In [15]:
%%time
# this seems a better model than the one above
#auto update alpha
#training model
import multiprocessing
model = gensim.models.doc2vec.Doc2Vec(dm=0, dbow_words=1, size=100, alpha=0.025, min_alpha=0.025, min_count=1, workers=multiprocessing.cpu_count(), iter=100, window = 30)
model.build_vocab(docs)
model.train(docs, total_examples = words_count, epochs = model.iter)

Wall time: 14min 35s


In [17]:
# shows the similar words
print (model.wv.most_similar('cancer'))
print() 
# shows the learnt embedding
print (model.wv['cancer'])
print() 
# shows the similar docs with tag = #hpv
print (model.docvecs.most_similar("#hpv"))
print() 
#hashtag embeddings
print(model.docvecs['#cervical'])

[('cervical', 0.7103636860847473), (':', 0.5652252435684204), ('hpv', 0.5650304555892944), ('for', 0.5563048124313354), ('cancers', 0.5442872047424316), ('and', 0.5355117321014404), ('screening', 0.5271674394607544), ('prevent', 0.52569580078125), ('pathogenposse', 0.5254772901535034), ('rt', 0.5195286273956299)]

[-0.01450148 -0.4765738  -0.27967104 -0.21792474  0.20908529  0.26640809
  0.47182855  0.44003326  0.16612442  0.06861772 -0.2064383   0.63898283
  0.23104607 -0.21566872  0.42022917 -0.02735193  0.29281154  0.03050879
  0.03291376  0.27262101  0.33217353  0.07228537 -0.10965021 -0.1724453
  0.4806695   0.04843138 -0.48427922  0.27168617 -0.10165881  0.17472968
 -0.55431938  0.35294515  0.44892219 -0.20762593  0.01711269 -0.03736882
  0.17307073  0.5755471  -0.12753339  0.20625421 -0.02579602 -0.05845352
  0.52650571 -0.00497864  0.11514082 -0.21894194 -0.1525425   0.07640699
 -0.24041913  0.71906883  0.32044071  0.29416862 -0.11677121 -0.00918423
  0.44369221 -0.32470855 -0.

In [16]:
#assess model
origin = model.docvecs['#hpv'] 
word_sims = [('word', word, score) for word, score in model.most_similar([origin],topn=20)]
tag_sims = [('tag', tag, score) for tag, score in model.docvecs.most_similar([origin],topn=20)]
results = sorted((tag_sims + word_sims),key=lambda tup: -tup[2])
results[:20]

E:\conda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('tag', '#hpv', 0.9999999403953552),
 ('tag', '#cancer', 0.48357513546943665),
 ('word', 'kyga14', 0.45989274978637695),
 ('word', 'hpg', 0.4580444395542145),
 ('word', 'airway', 0.4499026834964752),
 ('word', 'ucriverside', 0.4479418098926544),
 ('word', 'steer', 0.43955087661743164),
 ('word', 'dawn', 0.4394335150718689),
 ('tag', '#tissue', 0.4309648871421814),
 ('word', 'riskawaresexed', 0.43005794286727905),
 ('word', 'teenaged', 0.4226871728897095),
 ('word', 'foot', 0.4179244339466095),
 ('word', 'hns', 0.4167706370353699),
 ('word', 'startre', 0.4166020452976227),
 ('word', 'slpeeps', 0.4163530468940735),
 ('tag', '#acthiv', 0.41599318385124207),
 ('word', 'proficiency', 0.4132997989654541),
 ('word', 'womencomments', 0.4129256010055542),
 ('word', 'johannesburg', 0.41246163845062256),
 ('word', 'geteducatedoncancerprevention', 0.41180163621902466)]

In [ ]:
#what we can do with the doc2vec model?
